# Customer Satisfaction Based on Response

# Necessary Imports

In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [138]:
%pylab
plt.style.use( 'ggplot' )

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


# Reading the Data as .csv 

In [139]:
consumer_dataframe = pd.read_csv( 'Consumer_Complaints.csv' , dtype = object)

# Mapping the Yes : 1 and No :  for Timely Response and Customer Disputed

In [140]:
consumer_dataframe['response_in_time']=consumer_dataframe['Timely response?'].map({'Yes':1,'No':0})
consumer_dataframe['satisfied']=consumer_dataframe['Consumer disputed?'].map({'Yes':1,'No':0})

# Creating all the 4 Condition
# --Customer Satisfied and Response In Time
# --Customer Satisfied and Response Not In Time
# --Customer Not Satisfied and Response Not In Time
# --Customer Not Satisfied and Response in Time

In [234]:
consumer_dataframe['satisfied_in_time']=((consumer_dataframe['response_in_time']==1) 
                                     & (consumer_dataframe['satisfied']==1))
consumer_dataframe['satisfied_not_in_time']=((consumer_dataframe['response_in_time']==0) 
                                             & (consumer_dataframe['satisfied']==1))
consumer_dataframe['not_satisfied_not_in_time']=((consumer_dataframe['response_in_time']==0) 
                                            & (consumer_dataframe['satisfied']==0))
consumer_dataframe['not_satisfied_in_time']=((consumer_dataframe['response_in_time']==1) 
                                             & (consumer_dataframe['satisfied']==0))

# Creating the Data Frames for all above connditions and Total Complaints

In [235]:

st_in_time = pd.DataFrame(consumer_dataframe.groupby([ 'Company']).sum()
                          [ 'satisfied_in_time' ])
st_in_time.reset_index(inplace= True)
not_st_in_time = pd.DataFrame(consumer_dataframe.groupby([ 'Company']).sum()
                          [ 'not_satisfied_in_time' ])
not_st_in_time.reset_index( inplace = True )
st_not_in_time = pd.DataFrame(consumer_dataframe.groupby([ 'Company']).sum()
                          [ 'satisfied_not_in_time' ])
st_not_in_time.reset_index( inplace = True )
not_st_not_in_time = pd.DataFrame(consumer_dataframe.groupby([ 'Company']).sum()
                          [ 'not_satisfied_not_in_time' ])
not_st_not_in_time.reset_index( inplace = True )
total=pd.DataFrame(consumer_dataframe.groupby([ 'Company']).count()['Complaint ID'])
total.reset_index(inplace= True)

# Categorizing Companies based on the above Data

In [236]:
good_comp = pd.DataFrame( consumer_dataframe.groupby([ 'Company' ]).sum() 
                        [ 'satisfied_in_time' ].sort_values( ascending = False )).head( 10 )
good_comp.reset_index( inplace = True )
bad_comp = pd.DataFrame( consumer_dataframe.groupby([ 'Company' ]).sum() 
                        [ 'not_satisfied_in_time' ].sort_values( ascending = False )).head( 10 )
bad_comp.reset_index( inplace = True )
worst_comp = pd.DataFrame( consumer_dataframe.groupby([ 'Company' ]).sum() 
                        [ 'not_satisfied_not_in_time' ].sort_values( ascending = False )).head( 10 )
worst_comp.reset_index( inplace = True )
average_comp = pd.DataFrame( consumer_dataframe.groupby([ 'Company' ]).sum() 
                        [ 'satisfied_not_in_time' ].sort_values( ascending = False )).head( 10 )
average_comp.reset_index( inplace = True )
total_comp=pd.DataFrame( consumer_dataframe.groupby([ 'Company' ]).count() 
                        [ 'Complaint ID' ].sort_values( ascending = False )).head(10)
total_comp.reset_index( inplace = True )

# Converting the Above Companies Data into List

In [237]:
bad_comp_list = bad_comp[ 'Company' ].tolist()
good_comp_list = good_comp[ 'Company' ].tolist()
worst_comp_list = worst_comp[ 'Company' ].tolist()
average_comp_list = average_comp[ 'Company' ].tolist()
total_comp_list = total_comp[ 'Company' ].tolist()

# Checking for the above list in the DataFrame and Creating a new DataFrame from it

In [238]:
company_good = pd.DataFrame(st_in_time[( st_in_time.Company.isin ( good_comp_list ))])
company_bad = pd.DataFrame(not_st_in_time[( not_st_in_time.Company.isin ( bad_comp_list ))])
company_worst = pd.DataFrame(not_st_not_in_time[( not_st_not_in_time.Company.isin ( worst_comp_list ))])
company_average = pd.DataFrame(st_not_in_time[( st_not_in_time.Company.isin ( average_comp_list ))])
company_total = pd.DataFrame(total[( total.Company.isin ( total_comp_list ))])

# Setting Index as Company for Joining

In [239]:
company_good.set_index(['Company'],inplace= True)
company_bad.set_index(['Company'],inplace= True)
company_worst.set_index(['Company'],inplace= True)
company_average.set_index(['Company'],inplace= True)
company_total.set_index(['Company'],inplace= True)

# Joining all Companies data into One DataFrame to plot

In [240]:
company_good=company_good.join(company_bad)

In [241]:
company_worst=company_worst.join(company_average)

In [242]:
company=company_good.join(company_worst)

In [246]:
company=company.join(company_total)

# Filling on the missed(NaN) values

In [247]:
company.fillna(value=0,inplace= True)

# Changing column names for better Understanding

In [262]:
company.columns=['Satisfied In Time','Not Satisfied even after Reponse in Time','Neither Responded Nor Satisfied Customer','Customer Satisfied Even after late Response','Total No of Complaint']

# Ploting a Bar Graph with the above data

In [263]:
ax = company.plot(kind = 'bar', title = "Customer Satisfaction",
                colormap = "Spectral", figsize = (15,10),legend = True, fontsize = 12)
legend = ax.legend()
legend.draggable(state=True)
ax.set_xlabel("Company",fontsize = 18)
ax.set_ylabel("Number of Complaints",fontsize = 18)
plt.show()